In [41]:
with open("input.txt", 'r') as f :
    file_lines = f.readlines()

file_lines

def sedd_getter(seed_line):
    numbors = [int(line) for line in seed_line.split(':')[1].replace("  "," ").replace("\n","").split(' ') if line!= '']
    numbarz = []
    for i in range(0, len(numbors), 2):
        numbarz.append(range(numbors[i], numbors[i]+numbors[i+1]))
    return numbarz

def process_map(idx, lines):
    names = lines[idx].split()[0]
    idx +=1
    dico = {}
    while idx<len(lines) and lines[idx]!='\n':
        current_line = lines[idx].replace('\n','').strip().split()
        dest_start, base_start, range_count = [int(numb) for numb in current_line]
        dico.update({(base_start,base_start+range_count) : dest_start - base_start})
        idx+=1
    
    return dico, (idx+1)



In [42]:
seeds = sedd_getter(file_lines[0])
print((seeds))

[range(630335678, 701491197), range(260178142, 385183563), range(1548082684, 2067859967), range(4104586697, 4135279673), range(1018893962, 1429853752), range(3570781652, 3615843762), range(74139777, 180146501), range(3262608046, 3476068197), range(3022784256, 3144777386), range(2138898608, 2175668592)]


In [43]:
maps = file_lines[2:]
dict_list = []
x=0
while (x < len(maps)):
    current_dict, x = process_map(x, maps)
    dict_list.append(current_dict)


In [44]:
dict_list

[{(3423361099, 3584538761): -446105836,
  (1524036300, 1564316920): 1940773183,
  (2583891002, 2866714384): -1304921699,
  (1796922321, 1967984297): 1969340699,
  (23002578, 175896945): 388883345,
  (442452799, 517453058): 122327491,
  (1454220354, 1524036300): 967165570,
  (3014668733, 3073346036): 333501147,
  (1975611534, 2013126303): -1071783221,
  (1396258054, 1454220354): 2010589129,
  (3171884304, 3423361099): 871606197,
  (2866714384, 3014668733): -1925371302,
  (1206586182, 1396258054): -117288751,
  (3584538761, 3603317630): -693421859,
  (517453058, 639780549): -517453058,
  (932395829, 1206586182): 1558806041,
  (0, 23002578): 388883345,
  (3073346036, 3171884304): 871606197,
  (671222912, 932395829): 2833867191,
  (2013126303, 2361056735): 60329189,
  (2361056735, 2428416227): 548839036,
  (3603317630, 4114980437): -2041524945,
  (2458166323, 2583891002): 307225900,
  (4114980437, 4294967296): -946797416,
  (2428416227, 2458166323): 710016698,
  (175896945, 442452799): -53

In [45]:
from tqdm import tqdm

def add_range(range_object, n):
    return range(range_object.start + n, range_object.stop + n)

def clean_seeds(seeds):
    if len(seeds)<=1:
        return seeds
    else:
        start, stop = seeds[0].start, seeds[0].stop
        sta, sto = seeds[1].start, seeds[1].stop
        if start in seeds[1] or stop in seeds[1] or sta in seeds[0] or sto in seeds[0]:
            new_seed = [range(min(start, sta), max(stop, sto))]
        else:
            new_seed = seeds[:2]
        
        return new_seed + clean_seeds(seeds[2:])

def apply_transf(seeds, transf):
    ranges_output = []

    for key,value in transf.items():
        a,b = key
        i= 0
        while i <len(seeds):
            current_seed = seeds[i]
            seed_low, seed_high = current_seed.start, current_seed.stop

            if seed_high > a and seed_low < b:
                if seed_low >= a and seed_high <= b:
                    ranges_output.append(add_range(current_seed, value))
                    seeds[i] = seeds[0]
                    seeds = seeds[1:]
                elif seed_low < a and seed_high > b:
                    ranges_output.append(add_range(range(a,b), value))
                    seeds[i] = range(seed_low, a)
                    seeds.append(range(b, seed_high))
                elif a > seed_low:
                    ranges_output.append(add_range(range(a,seed_high), value))
                    seeds[i] = range(seed_low, a)
                elif b < seed_high:
                    ranges_output.append(add_range(range(seed_low,b), value))
                    seeds[i] = range(b, seed_high)
                else:
                    print("outside")
                    i+=1
            else:
                i+=1
    output = seeds + ranges_output
    #print(output)
    return clean_seeds(output)

def get_location(seeds, list):
    for dic in list:
        seeds = apply_transf(seeds, dic)
        
    return seeds

loc_list=[]
min([ol.start for ol in get_location(seeds, dict_list)])

99751240